# Ship Detection

This notebook demonstrates how to use the geoai package for ship detection using a pre-trained model. 
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/ship_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [ ]:
# %pip install geoai-py

## Import libraries

In [ ]:
import geoai

## Download sample data

In [ ]:
raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/ships_dubai.tif"
)
raster_path = geoai.download_file(raster_url, "ships_dubai.tif")

## Visualize data

In [ ]:
geoai.view_raster(raster_url)

## Initialize model

In [ ]:
detector = geoai.ShipDetector()

## Generate masks

In [ ]:
output_path = "ships_dubai_masks.tif"

In [ ]:
masks_path = detector.generate_masks(
    raster_path,
    output_path=output_path,
    confidence_threshold=0.9,
    mask_threshold=0.7,
    overlap=0.25,
    chip_size=(256, 256),
    batch_size=4,
)

## Vectorize masks

In [ ]:
gdf = detector.vectorize_masks(
    output_path,
    output_path="ships_dubai_masks.geojson",
    confidence_threshold=0.8,
    min_object_area=100,
    max_object_size=10000,
)

## Visualize initial results

In [ ]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Calculate geometric properties

In [ ]:
gdf = geoai.add_geometric_properties(gdf)
gdf.head()

In [ ]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Filter results

In [ ]:
m = geoai.view_raster(raster_url, backend="ipyleaflet")
m

Use the drawing tool to select the area of interest. 

In [ ]:
aoi = m.user_rois

if aoi is None:

    aoi = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {},
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [55.133729, 25.110277],
                            [55.134072, 25.11393],
                            [55.134823, 25.115601],
                            [55.136025, 25.117116],
                            [55.137677, 25.118127],
                            [55.140145, 25.118787],
                            [55.142248, 25.11902],
                            [55.142012, 25.118243],
                            [55.140831, 25.116728],
                            [55.13948, 25.116903],
                            [55.137956, 25.116825],
                            [55.136132, 25.115543],
                            [55.13566, 25.114416],
                            [55.135467, 25.1136],
                            [55.135939, 25.112609],
                            [55.136218, 25.111657],
                            [55.13551, 25.110685],
                            [55.134373, 25.110102],
                            [55.133729, 25.110277],
                        ]
                    ],
                },
            }
        ],
    }

In [ ]:
import geopandas as gpd

In [ ]:
aoi_gdf = gpd.GeoDataFrame.from_features(aoi["features"], crs="EPSG:4326").to_crs(
    gdf.crs
)

Intersect the selected area with the vectorized masks to filter the results.

In [ ]:
gdf_filter = gdf[gdf.intersects(aoi_gdf.geometry[0])]
gdf_filter.head()

## Visualize final results

In [ ]:
geoai.view_vector_interactive(gdf_filter, column="confidence", tiles=raster_url)

## Save results

In [ ]:
gdf_filter.to_file("ships_dubai.geojson")

![image](https://github.com/user-attachments/assets/80df3827-8453-45b2-af89-21662fdf95a6)